In [35]:
import sqlite3
from sqlite3 import Error
import csv

DB_FILE_PATH = 'testSQLite.db'
CSV_FILE_PATH = 'C:/Users/HP/Downloads/indian-movie-face-database-imfdb/imfdb.csv'


def connect_to_db(db_file):
    """
    Connect to an SQlite database
    
    """
    sqlite3_conn = None

    try:
        sqlite3_conn = sqlite3.connect(db_file)
        return sqlite3_conn

    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()


In [36]:
def insert_values_to_table(table_name, csv_file_path):
    """
    
    Open a csv file, store its content in a list excluding header and insert the data from the list to db table
    
    
    """

    conn = connect_to_db(DB_FILE_PATH)

    if conn is not None:
        c = conn.cursor()

        # Create table if it is not exist
        c.execute('CREATE TABLE IF NOT EXISTS ' + table_name +
                  '(image         CHAR,'
                  'trait         VARCHAR)')


        # Read CSV file content
        values_to_insert = open_csv_file(csv_file_path)

        # Insert to table
        if len(values_to_insert) > 0:
            column_names, column_numbers = get_column_names_from_db_table(c, table_name)

            values_str = '?,' * column_numbers
            values_str = values_str[:-1]

            sql_query = 'INSERT INTO ' + table_name + '(' + column_names + ') VALUES (' + values_str + ')'

            c.executemany(sql_query, values_to_insert)
            conn.commit()

            print('SQL insert process finished')
        else:
            print('Nothing to insert')

        conn.close()

    else:
        print('Connection to database failed')


In [37]:
def open_csv_file(csv_file_path):
    """
    Open and read data from a csv file without headers (skipping the first row)
    
    """
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)

        data = list()
        for row in reader:
            data.append(row)

        return data


In [38]:
def get_column_names_from_db_table(sql_cursor, table_name):
    """
    Scrape the column names from a database table to a list and convert to a comma separated string, count the number
    of columns in a database table

    """

    table_column_names = 'PRAGMA table_info(' + table_name + ');'
    sql_cursor.execute(table_column_names)
    table_column_names = sql_cursor.fetchall()

    column_count = len(table_column_names)

    column_names = list()

    for name in table_column_names:
        column_names.append(name[1])

    return ', '.join(column_names), column_count


In [39]:
if __name__ == '__main__':
    insert_values_to_table('imfdb_temp', CSV_FILE_PATH)


SQL insert process finished
